<a href="https://colab.research.google.com/github/susan291-gifs/SussieAssignment/blob/main/SimpleConv1d_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import math
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

###Problem 1

In [ ]:
class SimpleInitializerConv2d:
    def __init__(self, sigma=0.01):
        self.sigma = sigma

    def W(self, F, C, FH, FW):
        return self.sigma * np.random.randn(F, C, FH, FW)

    def B(self, F):
        return np.zeros(F)

class Conv2d:
    def __init__(self, F, C, FH, FW, P, S, initializer=None, optimizer=None, activation=None):
        self.P = P
        self.S = S
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        self.W = self.initializer.W(F, C, FH, FW)
        self.B = self.initializer.B(F)

    def set_weights(self, W, B):
        self.W = W
        self.B = B

    def output_shape2d(self, H, W, pad_h, pad_w, FH, FW, S_h, S_w):
        OH = (H + 2 * pad_h - FH) // S_h + 1
        OW = (W + 2 * pad_w - FW) // S_w + 1
        return OH, OW

    def forward(self, X, debug=False):
        self.X = X
        N, C, H, W = self.X.shape
        F, C, FH, FW = self.W.shape
        OH, OW = self.output_shape2d(H, W, self.P, self.P, FH, FW, self.S, self.S)
        self.params = N, C, H, W, F, FH, FW, OH, OW
        A = np.zeros([N, F, OH, OW])
        self.X_pad = np.pad(self.X, ((0, 0), (0, 0), (self.P, self.P), (self.P, self.P)))

        for n in range(N):
            for ch in range(F):
                for row in range(OH):
                    for col in range(OW):
                        A[n, ch, row, col] = np.sum(self.X_pad[n, :, row*self.S:row*self.S+FH, col*self.S:col*self.S+FW] * self.W[ch, :, :, :]) + self.B[ch]

        if debug:
            return A
        else:
            return self.activation.forward(A) if self.activation else A

    def backward(self, dZ, debug=False):
        if debug:
            dA = dZ
        else:
            dA = self.activation.backward(dZ) if self.activation else dZ
        N, C, H, W, F, FH, FW, OH, OW = self.params
        dX = np.zeros(self.X_pad.shape)
        self.dW = np.zeros(self.W.shape)
        self.dB = np.zeros(self.B.shape)

        for n in range(N):
            for ch in range(F):
                for row in range(OH):
                    for col in range(OW):
                        dX[n, :, row*self.S:row*self.S+FH, col*self.S:col*self.S+FW] += dA[n, ch, row, col] * self.W[ch, :, :, :]
                        self.dW[ch, :, :, :] += dA[n, ch, row, col] * self.X_pad[n, :, row*self.S:row*self.S+FH, col*self.S:col*self.S+FW]

        if self.P > 0:
            dX = dX[:, :, self.P:-self.P, self.P:-self.P]

        for ch in range(F):
            self.dB[ch] = np.sum(dA[:, ch, :, :])

        self.optimizer.update(self)
        return dX


class ReLU:
    def forward(self, A):
        self.A = A
        return np.maximum(0, A)

    def backward(self, dZ):
        return dZ * (self.A > 0)


class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB

###Problem 2

In [ ]:
x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])

w = np.array([[[[ 0.,  0.,  0.],
                [ 0.,  1.,  0.],
                [ 0., -1.,  0.]]],

              [[[ 0.,  0.,  0.],
                [ 0., -1.,  1.],
                [ 0.,  0.,  0.]]]])

b = np.array([0., 0.])


conv_layer = Conv2d(F=2, C=1, FH=3, FW=3, P=0, S=1, initializer=SimpleInitializerConv2d(), optimizer=SGD(lr=0.01), activation=None)
conv_layer.set_weights(w, b)


output = conv_layer.forward(x)
print("Forward Output:\n", output)


delta = np.array([[[[ -4,  -4],
                    [ 10,  11]],

                   [[  1,  -7],
                    [  1, -11]]]])


dX = conv_layer.backward(delta)
print("Backward Output (Gradient w.r.t input):\n", dX)

Forward Output:
 [[[[-4. -4.]
   [-4. -4.]]

  [[ 1.  1.]
   [ 1.  1.]]]]
Backward Output (Gradient w.r.t input):
 [[[[  0.   0.   0.   0.]
   [  0.  -5.   4.  -7.]
   [  0.  13.  27. -11.]
   [  0. -10. -11.   0.]]]]


###Problem 3

In [ ]:
def calculate_output_size(Nh_in, Nw_in, Ph, Pw, Fh, Fw, Sh, Sw):
    """
    Calculate the output size after 2D convolution.

    Parameters:
    - Nh_in (int): Input height
    - Nw_in (int): Input width
    - Ph (int): Padding height
    - Pw (int): Padding width
    - Fh (int): Filter height
    - Fw (int): Filter width
    - Sh (int): Stride height
    - Sw (int): Stride width

    Returns:
    - Nh_out (int): Output height
    - Nw_out (int): Output width
    """
    Nh_out = math.floor((Nh_in + 2 * Ph - Fh) / Sh) + 1
    Nw_out = math.floor((Nw_in + 2 * Pw - Fw) / Sw) + 1

    return Nh_out, Nw_out

# Example usage
Nh_in = 6  # Input height
Nw_in = 6  # Input width
Ph = 0     # Padding height
Pw = 0     # Padding width
Fh = 3     # Filter height
Fw = 3     # Filter width
Sh = 1     # Stride height
Sw = 1     # Stride width

Nh_out, Nw_out = calculate_output_size(Nh_in, Nw_in, Ph, Pw, Fh, Fw, Sh, Sw)
print(f"Output Height: {Nh_out}, Output Width: {Nw_out}")

Output Height: 4, Output Width: 4


###Problem 4

In [ ]:
class MaxPool2D:
    def __init__(self, pool_size, stride):
        self.pool_size = pool_size
        self.stride = stride

    def forward(self, X):
        self.X = X
        N, C, H, W = X.shape
        pool_height, pool_width = self.pool_size
        stride_height, stride_width = self.stride

        out_height = (H - pool_height) // stride_height + 1
        out_width = (W - pool_width) // stride_width + 1

        out = np.zeros((N, C, out_height, out_width))
        self.max_indices = np.zeros((N, C, out_height, out_width, 2), dtype=int)

        for n in range(N):
            for c in range(C):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * stride_height
                        h_end = h_start + pool_height
                        w_start = j * stride_width
                        w_end = w_start + pool_width

                        pool_region = X[n, c, h_start:h_end, w_start:w_end]
                        out[n, c, i, j] = np.max(pool_region)
                        max_index = np.unravel_index(np.argmax(pool_region), pool_region.shape)
                        self.max_indices[n, c, i, j] = (h_start + max_index[0], w_start + max_index[1])

        return out

    def backward(self, dA):
        N, C, H, W = self.X.shape
        pool_height, pool_width = self.pool_size
        stride_height, stride_width = self.stride
        _, _, out_height, out_width = dA.shape

        dX = np.zeros_like(self.X)

        for n in range(N):
            for c in range(C):
                for i in range(out_height):
                    for j in range(out_width):
                        max_h, max_w = self.max_indices[n, c, i, j]
                        dX[n, c, max_h, max_w] += dA[n, c, i, j]

        return dX


x = np.array([[[[ 1,  2,  3,  4],
                [ 5,  6,  7,  8],
                [ 9, 10, 11, 12],
                [13, 14, 15, 16]]]])

maxpool = MaxPool2D(pool_size=(2, 2), stride=(2, 2))
output = maxpool.forward(x)
print("Forward Output:\n", output)


delta = np.array([[[[ 1,  2],
                    [ 3,  4]]]])

dX = maxpool.backward(delta)
print("Backward Output (Gradient w.r.t input):\n", dX)

Forward Output:
 [[[[ 6.  8.]
   [14. 16.]]]]
Backward Output (Gradient w.r.t input):
 [[[[0 0 0 0]
   [0 1 0 2]
   [0 0 0 0]
   [0 3 0 4]]]]


In [ ]:
class FlattenLayer:
    def __init__(self):
        pass

    def forward(self, X):
        self.shape = X.shape
        return X.reshape(len(X), -1)

    def backward(self, X):
        return X.reshape(self.shape)

def load_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
    x_test = x_test.reshape(-1, 28, 28, 1) / 255.0
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

def build_model():
    input_layer = Input(shape=(28, 28, 1))
    x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(input_layer)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output_layer = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

x_train, y_train, x_test, y_test = load_mnist()

model = build_model()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.1)


test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')

Epoch 1/10
844/844 [==============================] - 188s 212ms/step - loss: 0.1351 - accuracy: 0.9579 - val_loss: 0.0471 - val_accuracy: 0.9867
Epoch 2/10
844/844 [==============================] - 163s 194ms/step - loss: 0.0402 - accuracy: 0.9876 - val_loss: 0.0440 - val_accuracy: 0.9885
Epoch 3/10
844/844 [==============================] - 170s 201ms/step - loss: 0.0241 - accuracy: 0.9924 - val_loss: 0.0463 - val_accuracy: 0.9883
Epoch 4/10
844/844 [==============================] - 176s 209ms/step - loss: 0.0153 - accuracy: 0.9950 - val_loss: 0.0485 - val_accuracy: 0.9892
Epoch 5/10
844/844 [==============================] - 171s 202ms/step - loss: 0.0121 - accuracy: 0.9964 - val_loss: 0.0378 - val_accuracy: 0.9902
Epoch 6/10
844/844 [==============================] - 165s 195ms/step - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.0489 - val_accuracy: 0.9890
Epoch 7/10
844/844 [==============================] - 166s 197ms/step - loss: 0.0078 - accuracy: 0.9975 - val_loss: 0.0581 -